In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install fastapi
!pip install uvicorn
!pip install fastapi uvicorn pyngrok
!pip install python-multipart
!pip install --upgrade fastapi uvicorn pydantic
!pip install mlflow --quiet

In [ ]:
!pip show xformers

Name: xformers
Version: 0.0.28.post3
Summary: XFormers: A collection of composable Transformer building blocks.
Home-page: https://facebookresearch.github.io/xformers/
Author: Facebook AI Research
Author-email: oncall+xformers@xmail.facebook.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, torch
Required-by: 


In [ ]:
!pip install git+https://github.com/casper-hansen/AutoAWQ.git

  Cloning https://github.com/casper-hansen/AutoAWQ.git to /tmp/pip-req-build-zgymzqbs
  Running command git clone --filter=blob:none --quiet https://github.com/casper-hansen/AutoAWQ.git /tmp/pip-req-build-zgymzqbs
  Resolved https://github.com/casper-hansen/AutoAWQ.git to commit 79547665bdb27768a9b392ef375776b020acbf0c
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 44.6 MB/s eta 0:00:00
  Created wheel for autoawq: filename=autoawq-0.2.6+cu122-py3-none-any.whl size=100481 sha256=f8320e7a26993fa98b34eb909a8d06fd927758e7432c4d2d07a38424aa3bbb85
  Stored in directory: /tmp/pip-ephem-wheel-cache-41kszz8t/wheels/45/15/ff/b8ad64aaf278ee25afd82532190f636ef2c2cd1551d03d49ea
Successfully built autoawq


In [ ]:
!pip show torch

Name: torch
Version: 2.5.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, autoawq, bitsandbytes, fastai, peft, sentence-transformers, timm, torchaudio, torchvision, unsloth_zoo, xformers


In [ ]:
from pyngrok import ngrok

# Replace the string with your authtoken
!ngrok authtoken 2nmXJlQmG0i71MvuFJ19lgWkSxx_49kgmSfyyaXCdoHik2aRu

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import torch
import mlflow
from transformers import TextStreamer
from fastapi import FastAPI, HTTPException, File, UploadFile, Form
from pydantic import BaseModel
import uvicorn
import pandas as pd
import os
from pathlib import Path
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from pyngrok import ngrok
import nest_asyncio
import asyncio
from datasets import Dataset
from datasets import load_dataset
import shutil
from io import BytesIO
import tempfile
from datetime import datetime
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
import uvicorn
from urllib.parse import urlparse
from zoneinfo import ZoneInfo



nest_asyncio.apply()

app = FastAPI()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
quant_config = {
    "zero_point": True,
    "q_group_size": 128,
    "w_bit": 4,
    "version": "GEMM"
}


def get_latest_checkpoint(model_base_dir):
    latest_model_dir = None
    try:
        directories = [dir for dir in model_base_dir.iterdir() if dir.is_dir()]
        if directories:
            latest_model_dir = max(directories, key=os.path.getctime)
            print(f"Latest model directory found: {latest_model_dir}")
        else:
            print("No previous model directories found.")
    except Exception as e:
        print(f"Error checking for latest model directory: {e}")
    return latest_model_dir


def get_latest_model_dir(base_dir):
    try:
        all_subdirs = [os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
        latest_subdir = max(all_subdirs, key=os.path.getmtime)
        return latest_subdir
    except ValueError:
        raise HTTPException(status_code=404, detail="No model directories found")


def get_current_datetime_str():
    return datetime.now(tz=ZoneInfo('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S")


class SOAPNotesPipeline(mlflow.pyfunc.PythonModel):
  def load_context(self, context):

    # Load the model and tokenizer artifacts from MLflow
    model_path = context.artifacts["checkpoint"]

    self.model, self.tokenizer = FastLanguageModel.from_pretrained(model_path)

    # Ensure the model is ready for inference
    FastLanguageModel.for_inference(self.model)

  def predict(self, context, input_conversation):
      alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
      ### Instruction:
      Write a SOAP notes similar to Response for a given doctor-patient conversation.

      ### doctor-patient conversation:
      {}

      ### Response:
      {}"""

      # Format the input conversation
      inputs = self.tokenizer(
          [alpaca_prompt.format(input_conversation, "")],
          return_tensors="pt"
      )

      # Initialize TextStreamer
      text_streamer = TextStreamer(self.tokenizer)

      # Generate SOAP notes using the model
      outputs = self.model.generate(
          **inputs,
          streamer=text_streamer,
          max_new_tokens=512
      )

      # Decode the generated SOAP notes
      decoded_outputs = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
      return decoded_outputs


def train_fn(data):
  os.environ["WANDB_DISABLED"] = "true"
  os.environ["WANDB_MODE"] = "dryrun"

  model_base_dir = Path("/content/model/Tigma-Finetuned")
    # Check if the base directory exists, and create it if it does not
  if not model_base_dir.exists():
      model_base_dir.mkdir(parents=True, exist_ok=True)
      print(f"Created model directory at: {model_base_dir}")
  else:
      print(f"Model directory already exists at: {model_base_dir}")
  latest_model_dir = get_latest_checkpoint(model_base_dir)

  if latest_model_dir:
      print(f"Loading from latest checkpoint: {latest_model_dir}")
      model, tokenizer = FastLanguageModel.from_pretrained(
          str(latest_model_dir),
          max_seq_length=8192,
          load_in_4bit=True
      )

  else:
      print("No checkpoint found, loading default model ID.")
      model,tokenizer = FastLanguageModel.from_pretrained(
          model_name="unsloth/Meta-Llama-3.1-8B",
          max_seq_length=8192,
          load_in_4bit=True
      )

  model = FastLanguageModel.get_peft_model(
            model,
            r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
            target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                              "gate_proj", "up_proj", "down_proj",],
            lora_alpha = 16,
            lora_dropout = 0, # Supports any, but = 0 is optimized
            bias = "none",    # Supports any, but = "none" is optimized
            # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
            use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
            random_state = 3407,
            use_rslora = False,  # We support rank stabilized LoRA
            loftq_config = None, # And LoftQ
          )


  alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
  ### Instruction:
  Write a SOAP notes similar to Response for a given doctor-patient conversation.

  ### doctor-patient conversation:
  {}

  ### Response:
  {}"""
  EOS_TOKEN = tokenizer.eos_token
  def formatting_prompts_func(examples):
      # instructions = examples["instruction"]
      inputs       = examples["Conversation"]
      outputs      = examples["GPT Response"]
      texts = []
      for  input, output in zip(inputs, outputs):
          # Must add EOS_TOKEN, otherwise your generation will go on forever!
          text = alpaca_prompt.format(input, output) + EOS_TOKEN
          texts.append(text)
      return { "text" : texts, }

  dataset = load_dataset('csv', data_files=data)
  dataset = dataset.map(formatting_prompts_func, batched=True)

  os.environ['MLFLOW_TRACKING_USERNAME'] = "sreebalajih"
  os.environ['MLFLOW_TRACKING_PASSWORD'] = "7ed63082e243bd20ccdc154728ea5620f45dffb4"
  os.environ['MLFLOW_TRACKING_PROJECTNAME'] = "Trainingflow_ML"

  mlflow.set_tracking_uri(f'https://dagshub.com/' + os.environ['MLFLOW_TRACKING_USERNAME'] + '/' + os.environ['MLFLOW_TRACKING_PROJECTNAME'] + '.mlflow')
  mlflow.set_experiment("Training experiment")

  tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

  with mlflow.start_run(run_name="MLflow for training flow"):
      # mlflow.log_input(dataset, context="training")

      max_steps = 5      # instead of epochs
      learning_rate= 2e-4

      trainer=SFTTrainer(
          model=model,
          tokenizer=tokenizer,
          train_dataset=dataset['train'],
          dataset_text_field = "text",
          max_seq_length = 8192,
          dataset_num_proc = 2,
          packing = False, # Can make training 5x faster for short sequences.

          args = TrainingArguments(
              per_device_train_batch_size = 2,
              gradient_accumulation_steps = 8,
              warmup_steps = 5,
              max_steps = 10,
              learning_rate = learning_rate,
              fp16 = not is_bfloat16_supported(),
              bf16 = is_bfloat16_supported(),
              logging_steps = 1,
              optim = "adamw_8bit",
              weight_decay = 0.01,
              lr_scheduler_type = "linear",
              seed = 3407,
              output_dir = "outputs",
              report_to = "none"
          ),
      )


      trainer_stats = trainer.train()
      mlflow.log_param("max_steps", max_steps)
      mlflow.log_param("learning_rate", learning_rate)
      mlflow.log_metrics(trainer_stats.metrics)

      model_info = mlflow.pyfunc.log_model(
          artifact_path ="Physicians-Copilot",
          python_model = SOAPNotesPipeline(),
          artifacts = {"checkpoint" : "/content/outputs/checkpoint-10"},
          registered_model_name = "Tigma-Finetuned-Model"
      )

      run_id = mlflow.active_run().info.run_id
      model_uri = model_info.model_uri

      del model
      del tokenizer
      torch.cuda.empty_cache()
  return trainer_stats, model_uri

def save_as_16bit_model(latest_model_dir, tokenizer, save_path):
    try:
        # Load the base model with the LoRA adapter
        model, tokenizer = FastLanguageModel.from_pretrained(
            str(latest_model_dir),
            max_seq_length=8192,
            load_in_4bit=True
        )

        # Use the specialized method to save the model as 16-bit merged
        print(f"Saving model as 16-bit merged format at: {save_path}")
        model.save_pretrained_merged(save_path, tokenizer, save_method="merged_16bit")

        # Save the tokenizer as well
        tokenizer.save_pretrained(save_path)

        print(f"Model successfully saved as 16-bit in {save_path}")

    except Exception as e:
        print(f"Error during 16-bit saving: {e}")
        raise HTTPException(status_code=500, detail=f"Failed to save the model as 16-bit: {str(e)}")


@app.post("/train/")
async def train_model(dataset: UploadFile = File(...)):
    try:
        # Save the uploaded file to a temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix=".csv") as tmp:
            tmp.write(await dataset.read())
            tmp_path = tmp.name  # Save the path to use later

        # Call your training function with the path of the uploaded file
        training_result, run_id = train_fn(tmp_path)
        return {"training_result":training_result, "run_id":run_id}

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

    finally:
        # Cleanup: delete the temporary file
        if os.path.exists(tmp_path):
            os.unlink(tmp_path)
    torch.cuda.empty_cache()


@app.post("/convert_16bit/")
async def convert_model_to_16bit():
    try:
        model_base_dir = Path("/content/model/Tigma-Finetuned")
        if not model_base_dir.exists():
            raise HTTPException(status_code=404, detail="Model base directory does not exist.")

        latest_model_dir = get_latest_checkpoint(model_base_dir)
        if not latest_model_dir:
            raise HTTPException(status_code=404, detail="No model checkpoint found.")

        # Create new directory for saving the 16-bit model
        merged_16bit_dir = Path(f"/content/model/merged_16bit/")
        merged_16bit_dir.mkdir(parents=True, exist_ok=True)

        # Call the function to save the model as 16-bit
        save_as_16bit_model(latest_model_dir, tokenizer=None, save_path=merged_16bit_dir)

        return {"message": f"Merged 16-bit model saved at: {merged_16bit_dir}"}

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))



@app.post("/quantize-awq/")
async def quantize_awq():
    try:
        # Define model path
        model_base_dir = "/content/model/merged_16bit/"
        latest_model_dir = get_latest_model_dir(model_base_dir)

        # Load the model and tokenizer
        model = AutoAWQForCausalLM.from_pretrained(latest_model_dir, **{"low_cpu_mem_usage": True, "use_cache": False})
        model.to(device)

        tokenizer = AutoTokenizer.from_pretrained(latest_model_dir, trust_remote_code=True)

        # Perform the quantization
        model.quantize(tokenizer, quant_config=quant_config)

        # Define save path with date and time
        save_dir = f"final_model/Tigmaminds-Finetuned-AWQ-{get_current_datetime_str()}"

        # Save the quantized model
        os.makedirs(save_dir, exist_ok=True)
        model.save_pretrained(save_dir)
        tokenizer.save_pretrained(save_dir)

        return {"status": "Quantization and saving successful", "saved_model_dir": save_dir}

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Quantization failed: {e}")

In [ ]:
if __name__ == "__main__":
    try:
        public_url = ngrok.connect(8000)
        print(f"Public URL: {public_url}")
        uvicorn.run(app, host="0.0.0.0", port=8000)
    except KeyboardInterrupt:
        print("Shutting down server...")
        ngrok.disconnect(public_url)
        asyncio.get_event_loop().run_until_complete(asyncio.get_event_loop().shutdown_asyncgens())
        asyncio.get_event_loop().close()

Public URL: NgrokTunnel: "https://b0ce-34-133-44-187.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [1487]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     152.58.204.160:0 - "POST /train HTTP/1.1" 307 Temporary Redirect
Model directory already exists at: /content/model/Tigma-Finetuned
No previous model directories found.
No checkpoint found, loading default model ID.
==((====))==  Unsloth 2024.11.1: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.0. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py", l

Generating train split: 0 examples [00:00, ? examples/s]

...

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 23 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 8
\        /    Total batch size = 16 | Total steps = 10
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.836600
2,2.700800
3,2.860900
4,1.780000
5,2.712800
6,2.604500
7,1.496300
8,2.533100
9,1.961700
10,1.427000


2024/11/06 16:37:55 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2024-09-26; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'unsloth-zoo'}
2024/11/06 16:37:55 WARNING mlflow.utils.requirements_utils: Found autoawq version (0.2.6+cu122) contains a local version label (+cu122). MLflow logged a pip requirement for this package as 'autoawq==0.2.6' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/11/06 16:37:55 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.20.0+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torchvision==0.20.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version 

INFO:     152.58.204.160:0 - "POST /train/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1487]


In [ ]:
### INFERENCE

In [ ]:
!pip install mlflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 113.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.1/569.1 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.2 MB/s eta 0:00:00


In [ ]:
import mlflow
logged_model = "runs:/9999d153b1604b4e87fcabfeebba9c24/Physicians-Copilot"
print(logged_model)

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

runs:/9999d153b1604b4e87fcabfeebba9c24/Physicians-Copilot


==((====))==  Unsloth 2024.11.1: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.0. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-7' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py", l

In [ ]:
import mlflow
logged_model = "runs:/9999d153b1604b4e87fcabfeebba9c24/Physicians-Copilot"
print(logged_model)

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

runs:/9999d153b1604b4e87fcabfeebba9c24/Physicians-Copilot


MlflowException: Run '9999d153b1604b4e87fcabfeebba9c24' not found

In [ ]:
import mlflow
logged_model = 'mlflow-artifacts:/5d73f6ae368644eb86bb6e9bd287f47c/9999d153b1604b4e87fcabfeebba9c24/artifacts/Physicians-Copilot'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

MlflowException: When an mlflow-artifacts URI was supplied, the tracking URI must be a valid http or https URI, but it was currently set to file:///content/mlruns. Perhaps you forgot to set the tracking URI to the running MLflow server. To set the tracking URI, use either of the following methods:
1. Set the MLFLOW_TRACKING_URI environment variable to the desired tracking URI. `export MLFLOW_TRACKING_URI=http://localhost:5000`
2. Set the tracking URI programmatically by calling `mlflow.set_tracking_uri`. `mlflow.set_tracking_uri('http://localhost:5000')`

In [ ]:
import mlflow
logged_model = 'mlflow-artifacts:/5d73f6ae368644eb86bb6e9bd287f47c/9999d153b1604b4e87fcabfeebba9c24/artifacts/Physicians-Copilot'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

==((====))==  Unsloth 2024.11.1: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.0. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
import mlflow
logged_model = 'runs:/0cb577b53f504ec4abf1efbbf0763479/Physicians-Copilot'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

==((====))==  Unsloth 2024.11.1: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.0. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-13' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py", 

In [ ]:
con = "Hello,  I have upper abdominal pain that radiates to my back, eventually I start having severe gagging & vomit.  After the retching/gagging subsides, the pain subsides.  I have had my gall bladder removed several years ago.  Had an upper endoscopy in Feb. 2013,  had double hernia repair with mesh 3/20/13.  thought that was causing this pain but i started having it again about 2 weeks ago.  I have had this for over 2 years & there is a pattern that it comes and goes.  Any Ideas? Hi welcome to Chat Doctor forum.  Thanks for calling Chat Doctor. Forum.  You have upper abdominal pain, radiates to back, have gagging and vomit then pain subsided. Endoscopy done double hernia found and repaired with mesh, pain recurred 2 weeks back the pain comes and goes. Gall bladder removed several years ago.  It appears that the pain is due to gastric irritation due to irritant foods, like spicy foods, junk foods, and oily foods, and some foods which your stomach is allergic like milk and milk products.  Wishing for a quick and complete recovery.  Best regards."
print(loaded_model.predict(con))

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
      ### Instruction:
      Write a SOAP notes similar to Response for a given doctor-patient conversation.

      ### doctor-patient conversation:
      Hello,  I have upper abdominal pain that radiates to my back, eventually I start having severe gagging & vomit.  After the retching/gagging subsides, the pain subsides.  I have had my gall bladder removed several years ago.  Had an upper endoscopy in Feb. 2013,  had double hernia repair with mesh 3/20/13.  thought that was causing this pain but i started having it again about 2 weeks ago.  I have had this for over 2 years & there is a pattern that it comes and goes.  Any Ideas? Hi welcome to Chat Doctor forum.  Thanks for calling Chat Doctor. Forum.  You have upper abdominal pain, radiates to back, have gagging and vomit then pain subsided. Endoscopy done double

In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
model_path = '/content/model/merged_16bit/20241022_101534'
quant_path = 'Tigma-Finetuned-awq'
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }

In [ ]:
# Load model
model = AutoAWQForCausalLM.from_pretrained(
    model_path, **{"low_cpu_mem_usage": True, "use_cache": False}
)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
model.quantize(tokenizer, quant_config=quant_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/167 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


val.jsonl.zst:   0%|          | 0.00/471M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/214670 [00:00<?, ? examples/s]

AWQ: 100%|██████████| 32/32 [16:31<00:00, 30.99s/it]
